In [1]:
!pip install fancyimpute mrmr-selection

  Using cached fancyimpute-0.7.0.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Using cached mrmr_selection-0.2.8-py3-none-any.whl.metadata (6.6 kB)
  Using cached knnimpute-0.1.0.tar.gz (8.3 kB)
  Preparing metadata (setup.py) ... done
  Using cached nose-1.3.7-py3-none-any.whl.metadata (1.7 kB)
  Using cached category_encoders-2.6.4-py2.py3-none-any.whl.metadata (8.0 kB)
Using cached mrmr_selection-0.2.8-py3-none-any.whl (15 kB)
Using cached category_encoders-2.6.4-py2.py3-none-any.whl (82 kB)
Using cached nose-1.3.7-py3-none-any.whl (154 kB)
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=e4ad157e022ca015ff94872dcc6e620aebaf92f626b7cbed0f9549971c3919f4
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=506ee7eedb1e31f21966400e91d05eed273b2bbcd6b7aeaf8a4df2d8492d3b78
  Stored in dir

#Libraries

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, LogisticRegression, LassoCV, LinearRegression
from sklearn.feature_selection import mutual_info_classif, chi2, RFE
from mlxtend.feature_selection import SequentialFeatureSelector
from mrmr import mrmr_classif
from fancyimpute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler


import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [3]:
# Open and load the file from Drive
directory = '/content/drive/MyDrive/Colab Notebooks/Project/'
file_path = directory + 'Book2.csv'
with open(file_path) as f:
    data = pd.read_csv(f)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Columns: 188 entries, Samples to Val
dtypes: float64(185), int64(1), object(2)
memory usage: 138.2+ KB


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,Prostate cancer,AA,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,Prostate cancer,EA,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,Case control,EA,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,Case control,EA,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,Case control,EA,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0


In [4]:
df = data.copy()

# shuffling the df
shuffled_df = shuffle(df, random_state=42)

# Encoding catergorical data to numerical data
df['Race'] = df['Race'].map({'EA': 0, 'AA': 1})
df['Status'] = df['Status'].map({'Case control': 0, 'Prostate cancer': 1})

df.head()


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,15298,1,1,84.85,2.26,1.635,27.75,19.85,34.75,1.69,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,17585,1,0,87.40,3.18,1.330,23.70,14.20,20.10,2.76,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,17848,0,0,157.00,7.33,3.250,43.10,43.10,92.30,3.51,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,17893,0,0,59.30,1.86,1.270,18.40,11.30,13.80,1.16,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,17962,0,0,57.80,1.78,0.781,18.00,12.10,13.20,1.48,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0


In [5]:
# Check for NaN values
nan_check = df.isna().sum()
print(nan_check[nan_check > 0])

# Check total number of NaN values in df
total_nan = df.isna().sum().sum()
print("Total number of NaN values in the DataFrame (Before):", total_nan)

df.tail(10)

PC aa C34:4    1
PC aa C36:0    1
PC aa C36:1    1
PC aa C36:2    1
PC aa C36:3    1
              ..
Ser            1
Thr            2
Trp            1
Tyr            1
Val            1
Length: 160, dtype: int64
Total number of NaN values in the DataFrame (Before): 170


,Samples,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
84,20049,0,1,63.0,1.93,0.918,19.30,13.20,29.7,1.95,...,229.0,36.7,84.3,98.5,325.0,143.0,221.0,68.8,129.0,368.0
85,20051,0,1,90.3,2.41,1.470,26.10,14.80,22.3,1.79,...,307.0,40.1,123.0,82.7,209.0,134.0,167.0,81.1,101.0,340.0
86,20070,0,1,74.0,2.26,0.933,20.70,15.00,17.2,1.43,...,248.0,33.6,56.5,69.8,219.0,103.0,135.0,47.8,87.0,243.0
87,20078,0,1,109.5,2.36,1.635,35.75,16.85,24.7,1.87,...,108.0,10.5,65.0,37.2,65.4,50.6,NaN,23.5,31.7,107.0
88,20099,0,1,118.0,3.68,2.090,32.30,19.50,27.8,2.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,20103,0,1,137.0,3.00,2.390,47.20,23.50,41.5,2.44,...,207.0,22.9,99.1,68.5,200.0,98.0,111.0,60.4,63.1,213.0
90,20319,0,1,182.0,4.81,2.470,52.60,27.80,46.2,2.81,...,389.0,49.3,201.0,158.0,514.0,136.0,204.0,108.0,140.0,495.0
91,20323,0,1,150.0,3.46,2.260,55.00,29.70,61.0,3.81,...,312.0,29.5,143.0,117.0,330.0,94.6,155.0,87.4,75.2,348.0
92,20361,0,1,133.0,2.74,2.130,45.00,19.80,39.9,2.46,...,300.0,35.0,151.0,87.1,510.0,163.0,247.0,73.0,105.0,324.0
93,20752,0,1,212.0,6.19,2.730,55.10,30.60,40.8,3.61,...,355.0,27.9,170.0,113.0,328.0,153.0,187.0,84.3,86.6,450.0


In [6]:
df = df.drop(columns=['Samples'])

df2=df.copy()

In [7]:
# # Visualize missing data pattern
# msno.matrix(df)
# plt.show()

# # Check for any particular patterns in missing data
# msno.heatmap(df)
# plt.show()


In [8]:
# Split data into train (80%) and test (20%)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=43)

# # Remove some values from the training data for testing imputation
train_data_with_missing = train_data.copy()

columns_to_exclude = ['Status', 'Race']

# Get the list of columns where missing values will be introduced
columns_to_modify = train_data_with_missing.columns.difference(columns_to_exclude)

# Introduce missing values only in the selected columns
train_data_with_missing.loc[::10, columns_to_modify] = pd.NA


In [9]:
# # Split data into train (80%) and test (20%)
# train_data, test_data = train_test_split(df, test_size=0.2, random_state=43)

# # Copy the training data
# train_data_with_missing = train_data.copy()

# # Columns to exclude from missing value introduction
# columns_to_exclude = ['Status', 'Race']
# columns_to_modify = train_data_with_missing.columns.difference(columns_to_exclude)

# # Create a mask to keep track of where we introduce missing values
# introduced_missing_mask = pd.DataFrame(False, index=train_data_with_missing.index, columns=train_data_with_missing.columns)
# introduced_missing_mask.loc[::10, columns_to_modify] = True

# # Introduce missing values only in the selected columns (every 10th row)
# train_data_with_missing = train_data_with_missing.mask(introduced_missing_mask, pd.NA)


In [10]:
train_data_with_missing.shape

(75, 187)

In [11]:
train_data_with_missing.tail(20)

,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
4,0,0,57.8,1.78,0.781,18.0,12.1,13.2,1.48,5.43,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.8,115.0
84,0,1,63.0,1.93,0.918,19.3,13.2,29.7,1.95,6.61,...,229.0,36.7,84.3,98.5,325.0,143.0,221.0,68.8,129.0,368.0
79,1,1,99.4,2.52,2.920,29.3,17.1,20.4,1.80,6.73,...,345.0,46.0,100.0,127.0,276.0,145.0,193.0,88.7,138.0,374.0
34,0,0,99.1,2.84,1.680,26.9,24.7,47.8,1.94,11.90,...,296.0,37.5,171.0,96.4,320.0,140.0,205.0,91.7,80.2,298.0
65,1,0,165.0,4.64,3.030,51.0,32.3,43.5,2.79,8.40,...,238.0,29.0,109.0,84.1,235.0,117.0,158.0,121.0,90.2,308.0
75,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,0,1,114.0,2.75,1.240,30.3,22.2,34.5,2.26,9.99,...,251.0,45.5,136.0,104.0,395.0,119.0,182.0,70.2,103.0,409.0
35,0,1,99.9,3.52,1.740,33.3,26.1,31.4,1.83,9.00,...,331.0,46.8,131.0,114.0,327.0,164.0,236.0,87.9,132.0,381.0
92,0,1,133.0,2.74,2.130,45.0,19.8,39.9,2.46,11.40,...,300.0,35.0,151.0,87.1,510.0,163.0,247.0,73.0,105.0,324.0
30,0,0,124.0,3.51,2.570,37.5,22.7,30.9,1.39,5.53,...,337.0,31.4,168.0,88.8,436.0,169.0,179.0,75.4,94.7,299.0


#NaN Value Treatment

##MICE (Linear)

In [12]:
mice_linear_imputer = IterativeImputer(estimator=LinearRegression(), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_linear_imputed = mice_linear_imputer.fit_transform(train_data_with_missing)

[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 3.75
[IterativeImputer] Change: 1095.2150399344068, scaled tolerance: 3.273 
[IterativeImputer] Ending imputation round 2/15, elapsed time 12.58
[IterativeImputer] Change: 124.38472379260408, scaled tolerance: 3.273 
[IterativeImputer] Ending imputation round 3/15, elapsed time 18.62
[IterativeImputer] Change: 236.55391207788358, scaled tolerance: 3.273 
[IterativeImputer] Ending imputation round 4/15, elapsed time 29.84
[IterativeImputer] Change: 128.41557470982787, scaled tolerance: 3.273 
[IterativeImputer] Ending imputation round 5/15, elapsed time 35.05
[IterativeImputer] Change: 60.141006350429066, scaled tolerance: 3.273 
[IterativeImputer] Ending imputation round 6/15, elapsed time 36.84
[IterativeImputer] Change: 55.352016319428664, scaled tolerance: 3.273 
[IterativeImputer] Ending imputation round 7/15, elapsed time 37.72
[IterativeImputer] Change: 50.18064

In [13]:
# import numpy as np
# import pandas as pd
# from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
# from sklearn.impute import IterativeImputer
# from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import NearestNeighbors

# # Custom PMM function
# def predictive_mean_matching(predicted_missing, observed_values, n_matches=5):
#     """
#     Applies Predictive Mean Matching (PMM) to impute missing values.

#     Parameters:
#     - predicted_missing: Predicted values for missing entries (array-like)
#     - observed_values: Observed (non-missing) values (array-like)
#     - n_matches: Number of nearest matches to consider

#     Returns:
#     - Array of imputed values (from observed values)
#     """
#     # Ensure inputs are numpy arrays
#     predicted_missing = np.array(predicted_missing).reshape(-1, 1)
#     observed_values = np.array(observed_values).reshape(-1, 1)

#     # Fit NearestNeighbors on observed values
#     nn = NearestNeighbors(n_neighbors=n_matches, metric='euclidean')
#     nn.fit(observed_values)

#     # Find nearest neighbors for each predicted value
#     _, indices = nn.kneighbors(predicted_missing)

#     # Randomly pick one observed value from the nearest neighbors
#     imputed_values = np.array([np.random.choice(observed_values[neighbors]) for neighbors in indices])

#     return imputed_values.ravel()  # Flatten to a 1D array

# # Define IterativeImputer with Linear Regression
# mice_linear_imputer = IterativeImputer(
#     estimator=LinearRegression(),
#     tol=1e-4,
#     max_iter=15,
#     random_state=42,
#     verbose=2
# )

# # Perform initial imputation
# initial_imputed = mice_linear_imputer.fit_transform(train_data_with_missing)

# # Post-process with PMM for each column
# df_pmm_imputed = initial_imputed.copy()

# for col_idx in range(train_data_with_missing.shape[1]):
#     # Identify missing values in the current column
#     missing_mask = np.isnan(train_data_with_missing[:, col_idx])

#     if np.any(missing_mask):
#         # Extract observed and predicted values
#         observed_values = train_data_with_missing[~missing_mask, col_idx]
#         predicted_missing = initial_imputed[missing_mask, col_idx]

#         # Apply PMM
#         df_pmm_imputed[missing_mask, col_idx] = predictive_mean_matching(predicted_missing, observed_values)

# # Convert the imputed data to a DataFrame for easier inspection
# df_pmm_imputed = pd.DataFrame(df_pmm_imputed, columns=[f"Feature_{i}" for i in range(df_pmm_imputed.shape[1])])

# print("PMM-Imputed DataFrame:")
# print(df_pmm_imputed)


##Function for Correlation-Based Imputation

In [14]:
def correlation_based_imputation(df):
    for column in df.columns:
        # Check if there are missing values in the column
        if df[column].isnull().sum() > 0:

            # Find the most correlated feature with the current column
            correlated_feature = df.corr()[column].dropna().idxmax()
            # print(f"Imputing {column} using {correlated_feature}")

            # Impute missing values in the correlated feature using mean (or other methods)
            df[correlated_feature].fillna(df[correlated_feature].mean(), inplace=True)

            # Train linear regression model on non-missing rows
            non_missing_rows = df[df[column].notnull()]
            model = LinearRegression()
            model.fit(non_missing_rows[[correlated_feature]], non_missing_rows[column])

            # Predict missing values for rows where the current column has NaNs
            missing_rows = df[df[column].isnull()]
            if not missing_rows.empty:
                predicted_values = model.predict(missing_rows[[correlated_feature]])
                df.loc[df[column].isnull(), column] = predicted_values
    return df

df_corr_imputed = correlation_based_imputation(train_data_with_missing)


##KNN

In [15]:
mice_knn_imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=5, weights='uniform'), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_knn_imputed = mice_knn_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 0.16
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##Ridgde

In [16]:
mice_ridge_imputer = IterativeImputer(estimator=Ridge(alpha=1.0), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_ridge_imputed = mice_ridge_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 0.52
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##Lasso

In [17]:
mice_lasso_imputer = IterativeImputer(estimator=Lasso(alpha=0.1), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_lasso_imputed = mice_lasso_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 2.22
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##ElasticNet

In [18]:
mice_elasticnet_imputer = IterativeImputer(estimator=ElasticNet(alpha=1.0, l1_ratio=0.5), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_elasticnet_imputed = mice_elasticnet_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 2.29
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##XGBRegressor

In [19]:
mice_xgb_imputer = IterativeImputer(estimator=XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_xgb_imputed = mice_xgb_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 50.48
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##SVR

In [20]:
mice_svr_imputer = IterativeImputer(estimator=SVR(C=1.0, epsilon=0.1),tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_svr_imputed = mice_svr_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 0.37
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##BayesianRidge

In [21]:
mice_bayesianridge_imputer = IterativeImputer(estimator=BayesianRidge(), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_bayesianridge_imputed = mice_bayesianridge_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 4.24
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##DecisionTreeRegressor

In [22]:
mice_tree_imputer = IterativeImputer(estimator=DecisionTreeRegressor(max_depth=5), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_tree_imputed = mice_tree_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 1.46
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##GradientBoostingRegressor

In [23]:
mice_gb_imputer = IterativeImputer(estimator=GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_gb_imputed = mice_gb_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 110.70
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


##RandomForestRegressor

In [24]:
mice_rf_imputer = IterativeImputer(estimator=RandomForestRegressor(n_estimators=100, max_depth=10), tol=1e-4, max_iter=15, random_state=42, verbose=2)
df_rf_imputed = mice_rf_imputer.fit_transform(train_data_with_missing)


[IterativeImputer] Completing matrix with shape (75, 187)
[IterativeImputer] Ending imputation round 1/15, elapsed time 143.16
[IterativeImputer] Change: 0.0, scaled tolerance: 3.273 
[IterativeImputer] Early stopping criterion reached.


PolynomialRegression

In [25]:
from sklearn.preprocessing import PolynomialFeatures

# Define a custom estimator that uses polynomial regression
class PolynomialRegressionImputer:
    def __init__(self, degree=2):
        self.degree = degree
        self.poly = PolynomialFeatures(degree=self.degree)
        self.model = LinearRegression()

    def fit(self, X, y):
        # Transform features into polynomial terms
        X_poly = self.poly.fit_transform(X)
        self.model.fit(X_poly, y)
        return self

    def predict(self, X):
        # Transform features into polynomial terms for prediction
        X_poly = self.poly.transform(X)
        return self.model.predict(X_poly)

    # Implementing get_params to make the class compatible with scikit-learn
    def get_params(self, deep=True):
        return {"degree": self.degree}

    # Implementing set_params to make the class compatible with scikit-learn
    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self.poly = PolynomialFeatures(degree=self.degree)
        return self

# Apply MICE using polynomial regression
polynomial_imputer = IterativeImputer(estimator=PolynomialRegressionImputer(degree=2), max_iter=10, random_state=42)

# Perform imputation
df_poly_imputed = polynomial_imputer.fit_transform(train_data_with_missing)


##MICE results

In [26]:
# List of imputed data arrays and their respective names
imputed_data = [
    ('MICE Linear', df_linear_imputed),
    ('MICE Ridge', df_ridge_imputed),
    ('MICE ElasticNet', df_elasticnet_imputed),
    ('MICE XGB', df_xgb_imputed),
    ('MICE SVR', df_svr_imputed),
    ('MICE KNN', df_knn_imputed),
    ('MICE BayesianRidge', df_bayesianridge_imputed),
    ('MICE Lasso', df_lasso_imputed),
    ('MICE Tree', df_tree_imputed),
    ('MICE GB', df_gb_imputed),
    ('MICE RF', df_rf_imputed),
    ('Correlation-Based', df_corr_imputed),
    ('MICE Poly', df_poly_imputed),
]


# Function to calculate RMSE, print results, and return the imputed DataFrames
def calculate_rmse(original, imputed_data):
    results = {}

    for method, imputed_array in imputed_data:
        # Convert numpy array to DataFrame
        imputed_df = pd.DataFrame(imputed_array, columns=original.columns)

        # Ensure original and imputed arrays have the same shape
        assert original.shape == imputed_df.shape, "Original and imputed arrays must have the same shape"

        # Mask arrays to exclude NaN values
        mask = ~np.isnan(original.values)

        # Compute RMSE only for non-NaN entries
        rmse = np.sqrt(mean_squared_error(original.values[mask], imputed_df.values[mask]))

        print(f"{method} Imputation RMSE: {rmse:.4f}")

        # Store the DataFrame and RMSE in the results dictionary
        results[method] = {
            'imputed_df': imputed_df,
            'rmse': rmse
        }

    return results


imputation_results = calculate_rmse(train_data, imputed_data)


MICE Linear Imputation RMSE: 82.2814
MICE Ridge Imputation RMSE: 84.6411
MICE ElasticNet Imputation RMSE: 84.6411
MICE XGB Imputation RMSE: 84.6411
MICE SVR Imputation RMSE: 84.6411
MICE KNN Imputation RMSE: 84.6411
MICE BayesianRidge Imputation RMSE: 84.6411
MICE Lasso Imputation RMSE: 84.6411
MICE Tree Imputation RMSE: 84.6411
MICE GB Imputation RMSE: 84.6411
MICE RF Imputation RMSE: 84.6411
Correlation-Based Imputation RMSE: 84.6411
MICE Poly Imputation RMSE: 84.6411


# FIlling NaN values

##Loop

In [27]:
import os
import pandas as pd
from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor

imputer_estimators = [
    ('MICE Linear', LinearRegression()),
    ('MICE Ridge', Ridge(alpha=1.0)),
    ('MICE ElasticNet', ElasticNet(alpha=1.0, l1_ratio=0.5)),
    ('MICE XGB', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)),
    ('MICE SVR', SVR(C=1.0, epsilon=0.1)),
    ('MICE KNN', KNeighborsRegressor(n_neighbors=5, weights='uniform')),
    ('MICE BayesianRidge', BayesianRidge()),
    ('MICE Lasso', Lasso(alpha=0.1)),
    ('MICE Tree', DecisionTreeRegressor(max_depth=5)),
    ('MICE GB', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3)),
    ('MICE RF', RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42))
]


results = {}


tol=1e-5
max_iter=50

# Iterate over all imputers
for name, estimator in imputer_estimators:
    print(f"\nRunning {name} imputer...")

    # Create and fit the imputer
    imputer = IterativeImputer(estimator=estimator, tol=tol, max_iter=max_iter, random_state=42, verbose=2)
    imputed_data = imputer.fit_transform(df)
    imputed_df = pd.DataFrame(imputed_data, columns=df.columns)

    # Save the imputed dataframe
    file_path = os.path.join(directory, f"clean_data_{name.lower().replace(' ', '_')}.xlsx")
    imputed_df.to_excel(file_path, index=False)

    # Check for NaN values
    nan_check = imputed_df.isna().sum()
    total_nan = nan_check.sum()

    # Check for negative values
    negative_counts = (imputed_df < 0).sum()
    total_negatives = negative_counts.sum()

    # Store results
    results[name] = {
        "total_nan": total_nan,
        "total_negatives": total_negatives,
        "columns_with_negatives": negative_counts[negative_counts > 0]
    }
    print(f"{name} - Total NaN: {total_nan}, Total Negatives: {total_negatives}")

# Print the final summary
for name, result in results.items():
    print(f"\n{name}:")
    print(f"Total NaN: {result['total_nan']}")
    print(f"Total Negatives: {result['total_negatives']}")
    if not result['columns_with_negatives'].empty:
        print("Columns with negatives:")
        print(result['columns_with_negatives'])



Running MICE Linear imputer...
[IterativeImputer] Completing matrix with shape (94, 187)
[IterativeImputer] Ending imputation round 1/50, elapsed time 1.29
[IterativeImputer] Change: 3678.884876702153, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 2/50, elapsed time 2.45
[IterativeImputer] Change: 356.80023390319025, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 3/50, elapsed time 3.64
[IterativeImputer] Change: 77.12367634043719, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 4/50, elapsed time 4.83
[IterativeImputer] Change: 58.716012075468804, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 5/50, elapsed time 7.98
[IterativeImputer] Change: 45.265470433988625, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 6/50, elapsed time 12.31
[IterativeImputer] Change: 35.828522434870536, scaled tolerance: 0.339030000000

In [49]:
mice_knn_imputer = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=5, weights='uniform'), tol=1e-5, max_iter=50, random_state=42, verbose=2)
df_knn_imputed = mice_knn_imputer.fit_transform(df)

[IterativeImputer] Completing matrix with shape (94, 187)
[IterativeImputer] Ending imputation round 1/50, elapsed time 0.27
[IterativeImputer] Change: 389.59390906965876, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 2/50, elapsed time 0.55
[IterativeImputer] Change: 133.0799999999999, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 3/50, elapsed time 0.85
[IterativeImputer] Change: 5.059999999999999, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Ending imputation round 4/50, elapsed time 1.12
[IterativeImputer] Change: 0.0, scaled tolerance: 0.33903000000000005 
[IterativeImputer] Early stopping criterion reached.


In [50]:
imputed_df_knn = pd.DataFrame(df_knn_imputed, columns=df.columns)

imputed_df = imputed_df_knn.copy()
imputed_df.tail(20)

,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
74,0.0,1.0,54.3,0.827,0.828,18.30,12.00,43.7,1.20,7.64,...,171.0,31.70,75.80,68.10,230.0,103.0,141.0,59.10,72.20,255.0
75,1.0,0.0,159.0,4.300,3.930,54.50,39.50,51.4,4.19,11.20,...,268.0,38.80,126.00,111.00,357.0,189.0,194.0,83.30,91.70,345.0
76,1.0,0.0,136.0,3.750,2.300,42.30,27.70,60.0,4.50,10.50,...,258.0,29.00,96.70,94.70,455.0,112.0,191.0,71.40,91.10,290.0
77,1.0,1.0,118.0,2.930,1.740,30.40,13.50,20.7,2.22,7.49,...,302.0,50.10,112.00,133.00,288.0,149.0,162.0,117.00,142.00,509.0
78,1.0,1.0,151.0,4.080,2.100,40.20,24.70,48.4,3.04,12.00,...,283.0,32.70,96.40,107.00,272.0,147.0,146.0,97.70,140.00,416.0
79,1.0,1.0,99.4,2.520,2.920,29.30,17.10,20.4,1.80,6.73,...,345.0,46.00,100.00,127.00,276.0,145.0,193.0,88.70,138.00,374.0
80,0.0,1.0,106.0,2.280,1.950,36.60,23.10,44.7,2.98,11.10,...,213.0,31.20,82.70,69.00,225.0,101.0,140.0,50.10,59.50,256.0
81,0.0,1.0,83.6,2.660,1.200,22.60,15.00,24.6,1.99,6.25,...,281.0,38.50,84.70,98.60,297.0,130.0,186.0,60.90,122.00,269.0
82,1.0,1.0,86.6,1.950,1.920,29.50,15.60,24.3,1.98,15.00,...,539.0,49.70,264.00,150.00,326.0,117.0,186.0,138.00,144.00,552.0
83,1.0,1.0,215.0,5.010,2.780,74.30,31.80,43.9,5.49,22.30,...,421.0,48.50,209.00,126.00,293.0,174.5,227.5,115.50,141.00,439.5


In [51]:
# # Check for NaN values
nan_check = imputed_df.isna().sum()
print(nan_check[nan_check > 0])

# Check total number of NaN values in df
total_nan = imputed_df.isna().sum().sum()
print("Total number of NaN values in the DataFrame (after):", total_nan)

Series([], dtype: int64)
Total number of NaN values in the DataFrame (after): 0


In [52]:
# Count the number of negative values in each column
negative_counts = (imputed_df < 0).sum()

# Show only the columns that have negative values
columns_with_negatives = negative_counts[negative_counts > 0]

print(columns_with_negatives)
total_negatives = negative_counts.sum()
print(total_negatives)

Series([], dtype: int64)
0


##Save imputed data

In [53]:
file_path = os.path.join(directory, 'clean_data_mice_knn.xlsx')
imputed_df.to_excel(file_path, index=False)

In [54]:
# file_path = directory + 'clean_data_mice_knn.xlsx'

# imputed_df = pd.read_excel(file_path)
imputed_df

,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,1.0,1.0,84.85,2.26,1.635,27.75,19.85,34.75,1.69,11.25,...,383.5,32.5,127.0,96.5,272.0,131.0,226.0,75.9,90.05,255.0
1,1.0,0.0,87.40,3.18,1.330,23.70,14.20,20.10,2.76,10.20,...,252.0,26.1,118.0,94.6,253.0,79.6,146.0,80.7,105.00,294.0
2,0.0,0.0,157.00,7.33,3.250,43.10,43.10,92.30,3.51,12.00,...,402.0,58.9,196.0,141.0,454.0,218.0,341.0,86.1,151.00,302.0
3,0.0,0.0,59.30,1.86,1.270,18.40,11.30,13.80,1.16,4.42,...,328.0,30.9,124.0,93.0,365.0,131.0,215.0,94.1,101.00,391.0
4,0.0,0.0,57.80,1.78,0.781,18.00,12.10,13.20,1.48,5.43,...,98.8,13.4,57.9,39.6,130.0,41.5,58.5,27.2,35.80,115.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.0,1.0,137.00,3.00,2.390,47.20,23.50,41.50,2.44,10.60,...,207.0,22.9,99.1,68.5,200.0,98.0,111.0,60.4,63.10,213.0
90,0.0,1.0,182.00,4.81,2.470,52.60,27.80,46.20,2.81,11.50,...,389.0,49.3,201.0,158.0,514.0,136.0,204.0,108.0,140.00,495.0
91,0.0,1.0,150.00,3.46,2.260,55.00,29.70,61.00,3.81,15.80,...,312.0,29.5,143.0,117.0,330.0,94.6,155.0,87.4,75.20,348.0
92,0.0,1.0,133.00,2.74,2.130,45.00,19.80,39.90,2.46,11.40,...,300.0,35.0,151.0,87.1,510.0,163.0,247.0,73.0,105.00,324.0


#Normalization

In [55]:
target_columns = ['Status', 'Race']
feature_columns = imputed_df.drop(columns=target_columns).columns

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

normalized_features = scaler.fit_transform(imputed_df[feature_columns])

normalized_features_df = pd.DataFrame(normalized_features, columns=feature_columns, index=imputed_df.index)

final_df = pd.concat([imputed_df[target_columns], normalized_features_df], axis=1)

file_path = os.path.join(directory, 'normalized_data.xlsx')
# file_path = os.path.join(directory, 'normalized_data_rf.xlsx')
final_df.to_excel(file_path, index=False)

In [57]:
final_df.info()
final_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Columns: 187 entries, Status to Val
dtypes: float64(187)
memory usage: 137.5 KB


,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,...,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,0.468085,0.500000,0.278085,0.251305,0.062796,0.216984,0.356632,0.275587,0.166923,0.327412,...,0.412825,0.356142,0.334868,0.402707,0.487083,0.460478,0.426075,0.477970,0.428018,0.464004
std,0.501656,0.502681,0.171984,0.145843,0.140148,0.165149,0.227045,0.184407,0.134089,0.177615,...,0.180617,0.159827,0.200550,0.174903,0.187861,0.197138,0.183624,0.203902,0.193812,0.197657
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.159546,0.157776,0.021572,0.117841,0.182632,0.144437,0.096557,0.197470,...,0.290663,0.263140,0.185422,0.285450,0.367181,0.325858,0.308099,0.352838,0.297778,0.332584
50%,0.000000,0.500000,0.261237,0.223765,0.035333,0.177863,0.353583,0.230088,0.138326,0.310790,...,0.400273,0.329876,0.296386,0.388441,0.478597,0.472296,0.431338,0.472926,0.401704,0.439326
75%,1.000000,1.000000,0.354139,0.309758,0.051708,0.270099,0.479751,0.381163,0.198329,0.421786,...,0.504203,0.446750,0.455422,0.501008,0.595876,0.588391,0.523768,0.594323,0.548083,0.583708
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [58]:
final_df.head(40)

,Status,Race,lysoPC a C16:0,lysoPC a C16:1,lysoPC a C17:0,lysoPC a C18:0,lysoPC a C18:1,lysoPC a C18:2,lysoPC a C20:3,lysoPC a C20:4,...,Lys,Met,Orn,Phe,Pro,Ser,Thr,Trp,Tyr,Val
0,1.0,1.0,0.135959,0.131794,0.026262,0.127203,0.275701,0.272440,0.083199,0.429530,...,0.646751,0.326418,0.339759,0.398522,0.403831,0.472296,0.595070,0.457642,0.355143,0.332584
1,1.0,0.0,0.147308,0.216408,0.016882,0.082599,0.099688,0.087231,0.173946,0.375323,...,0.348024,0.237898,0.296386,0.385753,0.366693,0.201055,0.313380,0.499563,0.446135,0.420225
2,0.0,0.0,0.457054,0.598087,0.075925,0.296256,1.000000,1.000000,0.237554,0.468250,...,0.688778,0.691563,0.672289,0.697581,0.759578,0.931398,1.000000,0.546725,0.726111,0.438202
3,0.0,0.0,0.022252,0.095006,0.015037,0.024229,0.009346,0.007585,0.038250,0.076923,...,0.520672,0.304288,0.325301,0.375000,0.585614,0.472296,0.556338,0.616594,0.421789,0.638202
4,0.0,0.0,0.015576,0.087648,0.000000,0.019824,0.034268,0.000000,0.065389,0.129066,...,0.000000,0.062241,0.006747,0.016129,0.126271,0.000000,0.005282,0.032314,0.024954,0.017978
5,1.0,1.0,0.283489,0.195254,0.050401,0.204846,0.398754,0.362832,0.198541,0.375323,...,0.543389,0.611342,0.412048,0.596774,0.632525,0.672823,0.609155,0.593013,0.939136,0.683146
6,0.0,1.0,0.007566,0.066495,0.005504,0.042952,0.000000,0.015171,0.000000,0.034073,...,0.123126,0.120332,0.174458,0.180108,0.155590,0.093931,0.041197,0.173799,0.200852,0.107865
7,0.0,1.0,0.128616,0.103283,0.027030,0.162996,0.149533,0.085967,0.083199,0.224058,...,0.066333,0.000000,0.140241,0.091398,0.016419,0.079683,0.000000,0.030568,0.026172,0.092135
8,0.0,1.0,0.159324,0.129035,0.029490,0.132159,0.236760,0.040455,0.083199,0.185854,...,0.200363,0.139696,0.291566,0.168011,0.190774,0.231662,0.190141,0.098690,0.146074,0.116854
9,0.0,0.0,1.000000,0.557620,0.075617,0.713656,0.959502,0.744627,0.497922,0.685080,...,0.457065,0.283541,0.677108,0.482527,0.466380,0.525066,0.584507,0.513537,0.404139,0.447191


In [59]:
df_filtered=final_df.copy()

# List of columns to exclude
columns_to_exclude = ['Status', 'Race']

# Select all columns except the specified ones
df_filtered = df_filtered.drop(columns=columns_to_exclude)

summary_stats = df_filtered.describe().T
skewness = df_filtered.skew()
summary_stats['Skewness'] = skewness

file_path = os.path.join(directory, 'data_skewness.xlsx')
summary_stats.to_excel(file_path, index=False)
summary_stats

,count,mean,std,min,25%,50%,75%,max,Skewness
lysoPC a C16:0,94.0,0.278085,0.171984,0.0,0.159546,0.261237,0.354139,1.0,1.155342
lysoPC a C16:1,94.0,0.251305,0.145843,0.0,0.157776,0.223765,0.309758,1.0,1.991072
lysoPC a C17:0,94.0,0.062796,0.140148,0.0,0.021572,0.035333,0.051708,1.0,5.453255
lysoPC a C18:0,94.0,0.216984,0.165149,0.0,0.117841,0.177863,0.270099,1.0,2.202366
lysoPC a C18:1,94.0,0.356632,0.227045,0.0,0.182632,0.353583,0.479751,1.0,0.648095
...,...,...,...,...,...,...,...,...,...
Ser,94.0,0.460478,0.197138,0.0,0.325858,0.472296,0.588391,1.0,0.054544
Thr,94.0,0.426075,0.183624,0.0,0.308099,0.431338,0.523768,1.0,0.388768
Trp,94.0,0.477970,0.203902,0.0,0.352838,0.472926,0.594323,1.0,0.111316
Tyr,94.0,0.428018,0.193812,0.0,0.297778,0.401704,0.548083,1.0,0.499780


In [60]:
final_df['Status'].value_counts()

,count
Status,
0.0,50
1.0,44


In [61]:
final_df['Race'].value_counts()

,count
Race,
1.0,47
0.0,47


# feature selection

Forward Feature Selection

In [38]:

# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a Forward Feature selection object
# ffs = SequentialFeatureSelector(rf, k_features=20, forward=True, n_jobs=-1)

# # Fit the Sequential Feature Selector to the data
# ffs.fit(X_train, y_train)

# # Get the selected features
# top_20_features = list(ffs.k_feature_names_)

# rf.fit(X_train[top_20_features], y_train)

# # Create a series of features
# feature_importances = pd.Series(rf.feature_importances_, index=top_20_features)

# # Sort features by importance score in descending order
# sorted_feature_importances = feature_importances.sort_values(ascending=False)

# # Plotting the top 20 features based on their importances in blue color
# plt.figure(figsize=(10, 6))
# sorted_feature_importances.plot(kind='barh', color='blue')
# plt.title('Top 20 Features Selected by Sequential Feature Selector')
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.show()

# # Predict on the test data and calculate accuracy using the selected features
# y_pred_ffs = rf.predict(X_test[top_20_features])
# accuracy_ffs = accuracy_score(y_test, y_pred_ffs)

# print("Accuracy using Forward Feature Selection top 20 features:", accuracy_ffs)
# print(sorted_feature_importances)


Backward Feature Selection

In [39]:
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a Backward Feature Elimination object
# # bfs = SequentialFeatureSelector(rf, k_features='best', forward=False, n_jobs=-1)
# bfs = SequentialFeatureSelector(rf, k_features=20, forward=False, n_jobs=-1)

# # Fit the Sequential Feature Selector to the data
# bfs.fit(X_train, y_train)

# # Get the selected features
# top_20_features_bfs = list(bfs.k_feature_names_)

# rf.fit(X_train[top_20_features_bfs], y_train)

# # Create a series of features importances
# feature_importances_bfs = pd.Series(rf.feature_importances_, index=top_20_features_bfs)

# # Sort features by importance score in descending order
# sorted_feature_importances_bfs = feature_importances_bfs.sort_values(ascending=False)

# # Plotting the top 20 features based on their importances in blue color
# plt.figure(figsize=(10, 6))
# sorted_feature_importances_bfs.plot(kind='barh', color='blue')
# plt.title('Top 20 Features Selected by Backward Feature Elimination')
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.show()

# # Predict on the test data and calculate accuracy using the selected features
# y_pred_bfs = rf.predict(X_test[top_20_features_bfs])
# accuracy_bfs = accuracy_score(y_test, y_pred_bfs)

# print("Accuracy using Backward Feature Elimination top 20 features:", accuracy_bfs)
# print(sorted_feature_importances_bfs)


In [40]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Create a Backward Feature Elimination object
# bfe = SequentialFeatureSelector(rf, k_features=20, forward=False, n_jobs=-1)

# # Fit the Sequential Feature Selector to the data
# bfe.fit(X_train, y_train)

# # Get the selected features
# selected_features = list(bfe.k_feature_names_)

# # Fit the model using the selected features
# rf.fit(X_train[selected_features], y_train)

# # Get feature importances
# feature_importances = pd.Series(rf.feature_importances_, index=selected_features)

# # Sort features by importance score in descending order
# sorted_feature_importances = feature_importances.sort_values(ascending=False)

# # Plotting the top features based on their importances
# plt.figure(figsize=(10, 6))
# sorted_feature_importances.plot(kind='barh', color='blue')
# plt.title('Feature Importances from Backward Feature Elimination')
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.show()

# # Predict on the test data
# y_pred_bfe = rf.predict(X_test[selected_features])
# accuracy_bfe = accuracy_score(y_test, y_pred_bfe)

# # Calculate the accuracy with selected features
# print("Accuracy using Backward Feature Elimination:", accuracy_bfe)
# print("Selected Features by Backward Feature Elimination:")
# print(sorted_feature_importances)
